In [1]:
import xlwings as xw
import os
import time
import datetime

# === CONFIGURATION ===
folder_path = r"xxx"
consolidation_path = r"zzz"
source_sheet_name = "yyy"
source_range = "B5:H2500"       # Fixed range to extract from each source file
clear_range_address = "A7:AB2500"  # Fixed range to clear in consolidation sheet
match_cell = "U3"             # Cell that holds the name to match
paste_start_cell = "A7"       # Where to paste the new data

# === START TIMER ===
start_time = time.time()

# === OPEN EXCEL AND CONSOLIDATION FILE ===
app = xw.App(visible=True)   # Start visible for first file
app.display_alerts = False
app.screen_updating = False

wb_consol = app.books.open(consolidation_path)

first_file = True

for filename in os.listdir(folder_path):
    if not filename.lower().endswith((".xlsx", ".xlsb")):
        continue
    if filename == os.path.basename(consolidation_path):
        continue

    filepath = os.path.join(folder_path, filename)
    file_basename = os.path.splitext(filename)[0]

    print(f"\n🔄 Processing: {filename}")

    wb_source = None
    try:
        wb_source = app.books.open(filepath, update_links=False)
        print(f"✅ Opened: {filename}")

        if source_sheet_name not in [s.name for s in wb_source.sheets]:
            print(f"⚠️ Sheet '{source_sheet_name}' not found in '{filename}'")
            continue

        sht_source = wb_source.sheets[source_sheet_name]
        sht_source.api.AutoFilterMode = False

        data = sht_source.range(source_range).value
        if isinstance(data[0], (str, float, int)) or data[0] is None:
            data = [data]  # Ensure 2D

        matched = False
        for sht_consol in wb_consol.sheets:
            if sht_consol.range(match_cell).value == file_basename:
                sht_consol.api.AutoFilterMode = False
                sht_consol.range(clear_range_address).clear_contents()
                sht_consol.range(paste_start_cell).value = data
                sht_consol.activate()

                if first_file:
                    # Select A1 to flush changes on first sheet
                    try:
                        sht_consol.range("A1").select()
                        time.sleep(1.0)
                    except Exception as e:
                        print(f"⚠️ Select failed: {e}")

                try:
                    wb_consol.macro("Recalculate")(sht_consol.name)
                    print(f"⚙️ Macro run on {sht_consol.name}")
                except Exception as macro_err:
                    print(f"❌ Macro failed: {macro_err}")

                matched = True
                break

        if not matched:
            print(f"⚠️ No matching sheet in consolidation for '{file_basename}'")

    except Exception as e:
        print(f"❌ Error processing {filename}: {e}")

    finally:
        if wb_source:
            wb_source.close()
            del wb_source
            print(f"📁 Closed: {filename}")

    if first_file:
        # Hide Excel after first file processed
        app.visible = False
        first_file = False

# === SAVE WITH TIMESTAMPED COPY ===
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
folder = os.path.dirname(consolidation_path)
base_name = os.path.splitext(os.path.basename(consolidation_path))[0]
new_filename = f"{base_name}_{timestamp}.xlsb"
new_path = os.path.join(folder, new_filename)

wb_consol.save(new_path)
wb_consol.close()
app.quit()

print(f"📁 Saved updated consolidation file as: {new_filename}")

# Open the new file for the user
os.startfile(new_path)
print(f"📂 Opened new file: {new_filename}")

# === END TIMER ===
end_time = time.time()
elapsed = end_time - start_time
print(f"\n🕒 Script completed in {elapsed:.2f} seconds.")



🔄 Processing: 1H IE Weight - Freeze 24-10-2025 - 4Q25 Fcst.xlsb
✅ Opened: 1H IE Weight - Freeze 24-10-2025 - 4Q25 Fcst.xlsb
⚠️ Sheet 'BPT format' not found in '1H IE Weight - Freeze 24-10-2025 - 4Q25 Fcst.xlsb'
📁 Closed: 1H IE Weight - Freeze 24-10-2025 - 4Q25 Fcst.xlsb

🔄 Processing: Accel - Freeze 24-10-2025 - 4Q25 Fcst.xlsb
✅ Opened: Accel - Freeze 24-10-2025 - 4Q25 Fcst.xlsb
⚙️ Macro run on Accel
📁 Closed: Accel - Freeze 24-10-2025 - 4Q25 Fcst.xlsb

🔄 Processing: DSR BRZ - Freeze 24-10-2025 - 4Q25 Fcst.xlsb
✅ Opened: DSR BRZ - Freeze 24-10-2025 - 4Q25 Fcst.xlsb
⚙️ Macro run on DSR BRZ
📁 Closed: DSR BRZ - Freeze 24-10-2025 - 4Q25 Fcst.xlsb

🔄 Processing: LD - Freeze 24-10-2025 - 4Q25 Fcst.xlsx
✅ Opened: LD - Freeze 24-10-2025 - 4Q25 Fcst.xlsx
⚠️ Sheet 'BPT format' not found in 'LD - Freeze 24-10-2025 - 4Q25 Fcst.xlsx'
📁 Closed: LD - Freeze 24-10-2025 - 4Q25 Fcst.xlsx

🔄 Processing: QtQ - Freeze 24-10-2025 - 4Q25 Fcst.xlsx
✅ Opened: QtQ - Freeze 24-10-2025 - 4Q25 Fcst.xlsx
⚙️ Macro 